# Title
[]()

In [90]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
from plotly.subplots import make_subplots
import requests
import json
import re

In [ ]:
# with open("credentials.json") as f:
#     credentials = json.load(f)

# ig_user_id = credentials['am_ig_user_id']
# access_token = credentials['am_ig_access_token']

In [3]:
with open("credentials.json") as f:
    credentials = json.load(f)

ig_user_id = credentials['ig_user_id']
access_token = credentials['access_token']

# `update_ig_account_insights`

## Version 1

In [7]:
def update_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    SH 2023-03-09 12:37
    Get the daily impressions and reach a given Instagram account. Load results that were previously saved.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    try:
        df = loadpickle(filename, csv_path)
        previous_since = df.sort_values('timestamp')
    except:
        pass
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    else:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    while (since_parameter >= since):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
                json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
        earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        print('Number of days of data:',len(df))
    except:
        df = response
    if filename:
        filename = f'{filename}_account_insights'
        try:
            savepickle(df, filename, 'sav', csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

since = '2023-02-01'
until = '2023-03-01'
filename = 'silvialiftsweights'

data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since=since, until=until, filename=filename)

https://graph.facebook.com/v15.0/17841401256046961/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1677657600.0&since=1675238400.0
Requesting page 1...
	Response status code:  200
Requesting page 2...
	Response status code:  200
Number of days of data: 57
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_account_insights.sav
Time completed: 2023-03-09 12:49:03.466033
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_account_insights.sav
Time completed: 2023-03-09 12:49:03.468023


## version 2

In [75]:

from datetime import time, datetime, timedelta
def update_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    SH 2023-03-09 12:37
    Get the daily impressions and reach a given Instagram account. Load results that were previously saved.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    if filename:
        filename = f'{filename}_account_insights'
    try:
        df = loadpickle(filename+'.sav', csv_path)
        response_json_dict = loadpickle(filename+'.sav', json_path)
        last_json_page = max(response_json_dict.keys())
        previous_since = response_json_dict[last_json_page]['data'][0]['values'][0]['end_time']
        previous_since = datetime.strptime(previous_since, "%Y-%m-%dT%H:%M:%S%z") # the %z format code is to indicate timezone as an offset
        
        previous_until = response_json_dict[1]['data'][0]['values'][-1]['end_time']
        previous_until = datetime.strptime(previous_until, "%Y-%m-%dT%H:%M:%S%z")
        print('previous until:', previous_until)

    except:
        pass
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if (previous_since.date() > since.date()) | (previous_until.date() < until.date()):
        print('Fetching new account insights')
    # if since_parameter:
    #     url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    # else:
    #     url_without_token += f'&since={datetime.timestamp(since)}'
    #     since_parameter = since

    # url = url_without_token+'&access_token='+access_token
    # print(url_without_token)
    
    # response_json_dict = dict()
    # df_list = []
    # earliest_end_time = None
    # page = 1
    # while (since_parameter >= since):
    #     response = requests.get(url)
    #     print(f'Requesting page {page}...')
    #     print('\tResponse status code: ',response.status_code)
    #     response_json_dict[page] = response.json()
    #     if response.status_code//100 != 2: # Stop the function if there is an error in the request
    #         print(response_json_dict[page]['error'])
    #         break
    #     try:
    #         df_list.append(
    #             pd.concat([
    #             json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
    #             json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
    #             ], axis=1)
    #         )
    #     except:
    #         print('No data in request response for page', page)
    #     earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
    #     since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")

    #     try:
    #         next_endpoint = response_json_dict[page]['paging']['previous']
    #         if next_endpoint+access_token != url:
    #             url = next_endpoint
    #         else:
    #             print('end')
    #             break
    #     except: 
    #         break
    #     page +=1
    # try:
    #     df = pd.concat(df_list)
    #     df = df.reset_index(drop=True)
    #     print('Number of days of data:',len(df))
    # except:
    #     df = response
    # if filename:
    #     try:
    #         savepickle(df, filename+'_df', 'sav', csv_path)
    #         savepickle(response_json_dict,filename,'sav',json_path)
    #     except:
    #         print('Unable to save outputs')
    return df, response_json_dict

since = '2023-02-01'
until = '2023-03-01'
filename = 'silvialiftsweights'

data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since=since, until=until, filename=filename)
# response_json_dict

# data = update_ig_account_insights(
#     ig_user_id, access_token, since=since, until=until, filename=filename)
# data

Time completed: 2023-03-09 13:57:31.405137
Time completed: 2023-03-09 13:57:31.407139
previous until: 2023-03-01 08:00:00+00:00


In [84]:
len(response_json_dict)

2

In [86]:
dict(zip([key+len(response_json_dict) for key in response_json_dict.keys()], response_json_dict.values())).keys()

dict_keys([3, 4])

In [89]:
{**response_json_dict, **dict(zip([key+len(response_json_dict) for key in response_json_dict.keys()], response_json_dict.values()))}.keys()

dict_keys([1, 2, 3, 4])

## Version 3

In [93]:
def get_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    2023-03-02 16:13
    Get the daily impressions and reach a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    else:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    while (since_parameter >= since):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
                json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
        earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        print('Number of days of data:',len(df))
    except:
        df = response
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

In [104]:
# Make GET request only for dates that have not been saved in previous queries
from datetime import time, datetime, timedelta
def update_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    SH 2023-03-09 12:37
    Get the daily impressions and reach a given Instagram account. Load results that were previously saved.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    if filename:
        filename = f'{filename}_account_insights'
    try:
        df = loadpickle(filename+'.sav', csv_path)
        response_json_dict = loadpickle(filename+'.sav', json_path)
        last_json_page = max(response_json_dict.keys())
        previous_since = response_json_dict[last_json_page]['data'][0]['values'][0]['end_time']
        previous_since = datetime.strptime(previous_since, "%Y-%m-%dT%H:%M:%S%z") # the %z format code is to indicate timezone as an offset
        
        previous_until = response_json_dict[1]['data'][0]['values'][-1]['end_time']
        previous_until = datetime.strptime(previous_until, "%Y-%m-%dT%H:%M:%S%z")
    except:
        pass
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if (previous_since.date() > since.date()):
        print(f'Fetching new account insights from {datetime.strftime(since, "%Y-%m-%d")} to {datetime.strftime(previous_since, "%Y-%m-%d")}')
        new_insights_df, new_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=since, until=previous_since)
        df = pd.concat([df, new_insights_df])
        new_insights_response_json_dict = dict( # Update the keys of new_insights_response_json_dict before merging with previous dict
            zip([key+len(response_json_dict) for key in new_insights_response_json_dict.keys()], new_insights_response_json_dict.values())
            )
        response_json_dict = {**response_json_dict, **new_insights_response_json_dict}
        
    return df, response_json_dict, new_insights_response_json_dict

since = '2023-02-01'
until = '2023-03-01'
filename = 'silvialiftsweights'

data, response_json_dict, new_insights_response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-01', until=until, filename=filename)


Time completed: 2023-03-09 14:37:34.968390
Time completed: 2023-03-09 14:37:34.968390
Fetching new account insights from 2023-01-01 to 2023-01-04
https://graph.facebook.com/v15.0/17841401256046961/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1672819200.0&since=1672560000.0
Requesting page 1...
	Response status code:  200
Requesting page 2...
	Response status code:  200
Number of days of data: 7


In [ ]:
new_insights_response_json_dict

In [ ]:
response_json_dict[4]

In [ ]:
response_json_dict[1]

## Version 4

In [3]:
def get_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    2023-03-02 16:13
    Get the daily impressions and reach a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    else:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    while (since_parameter >= since):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
                json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
        earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")
        print('since_parameter: ',since_parameter)

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        print('Number of days of data:',len(df))
    except:
        df = response
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

In [4]:
# Make GET request only for dates that have not been saved in previous queries
from datetime import time, datetime, timedelta
def update_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    SH 2023-03-09 12:37
    Get the daily impressions and reach a given Instagram account. Load results that were previously saved.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    if filename:
        filename = f'{filename}_account_insights'
    try:
        df = loadpickle(filename+'.sav', csv_path)
        response_json_dict = loadpickle(filename+'.sav', json_path)
        last_json_page = max(response_json_dict.keys())
        previous_since = response_json_dict[last_json_page]['data'][0]['values'][0]['end_time']
        previous_since = datetime.strptime(previous_since, "%Y-%m-%dT%H:%M:%S%z") # the %z format code is to indicate timezone as an offset
        
        previous_until = response_json_dict[1]['data'][0]['values'][-1]['end_time']
        previous_until = datetime.strptime(previous_until, "%Y-%m-%dT%H:%M:%S%z")
    except:
        pass
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if (previous_since.date() > since.date()):
        print(f'Fetching new account insights from {datetime.strftime(since, "%Y-%m-%d")} to {datetime.strftime(previous_since, "%Y-%m-%d")}')
        new_insights_df, new_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=since, until=previous_since)
        df = pd.concat([df, new_insights_df])
        new_insights_response_json_dict = dict( # Update the keys of new_insights_response_json_dict before merging with previous dict
            zip([key+len(response_json_dict) for key in new_insights_response_json_dict.keys()], new_insights_response_json_dict.values())
            )
        response_json_dict = {**response_json_dict, **new_insights_response_json_dict}
        
    # if (previous_until.date() < until.date()):

    # if since_parameter:
    #     url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    # else:
    #     url_without_token += f'&since={datetime.timestamp(since)}'
    #     since_parameter = since

    # url = url_without_token+'&access_token='+access_token
    # print(url_without_token)
    
    # response_json_dict = dict()
    # df_list = []
    # earliest_end_time = None
    # page = 1
    # while (since_parameter >= since):
    #     response = requests.get(url)
    #     print(f'Requesting page {page}...')
    #     print('\tResponse status code: ',response.status_code)
    #     response_json_dict[page] = response.json()
    #     if response.status_code//100 != 2: # Stop the function if there is an error in the request
    #         print(response_json_dict[page]['error'])
    #         break
    #     try:
    #         df_list.append(
    #             pd.concat([
    #             json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
    #             json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
    #             ], axis=1)
    #         )
    #     except:
    #         print('No data in request response for page', page)
    #     earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
    #     since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")

    #     try:
    #         next_endpoint = response_json_dict[page]['paging']['previous']
    #         if next_endpoint+access_token != url:
    #             url = next_endpoint
    #         else:
    #             print('end')
    #             break
    #     except: 
    #         break
    #     page +=1
    # try:
    #     df = pd.concat(df_list)
    #     df = df.reset_index(drop=True)
    #     print('Number of days of data:',len(df))
    # except:
    #     df = response
    # if filename:
    #     try:
    #         savepickle(df, filename+'_df', 'sav', csv_path)
    #         savepickle(response_json_dict,filename,'sav',json_path)
    #     except:
    #         print('Unable to save outputs')
    return df, response_json_dict, new_insights_response_json_dict

since = '2023-01-01'
until = '2023-03-01'
filename = 'silvialiftsweights'

data, response_json_dict, new_insights_response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-01', until=until, filename=filename)


Time completed: 2023-03-14 13:10:43.161301
Time completed: 2023-03-14 13:10:43.180351
Fetching new account insights from 2023-01-01 to 2023-01-04
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1672819200.0&since=1672560000.0
Requesting page 1...
	Response status code:  200
since_parameter:  2023-01-01 00:00:00
Requesting page 2...
	Response status code:  200
since_parameter:  2022-12-29 00:00:00
Number of days of data: 7


## Version 5

In [21]:
def get_ig_account_insights(ig_user_id, access_token, since=None, until=None, since_parameter=None,
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    2023-03-02 16:13
    Get the daily impressions and reach a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter < since:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    elif since_parameter == None:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since
    else:
        pass

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    print('since parameter:', since_parameter)
    print('since:', since)
    while (since_parameter > since):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
                json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
        earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")
        print('since_parameter: ',since_parameter)

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        print('Number of days of data:',len(df))
    except:
        df = df_list 
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

In [22]:
# Make GET request only for dates that have not been saved in previous queries
from datetime import time, datetime, timedelta
def update_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    SH 2023-03-09 12:37
    Get the daily impressions and reach a given Instagram account. Load results that were previously saved.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    if filename:
        filename = f'{filename}_account_insights'
    try:
        df = loadpickle(filename+'.sav', csv_path)
        response_json_dict = loadpickle(filename+'.sav', json_path)
        last_json_page = max(response_json_dict.keys())
        previous_since_tz = response_json_dict[last_json_page]['data'][0]['values'][0]['end_time']
        # previous_since_tz = datetime.strptime(previous_since_tz, "%Y-%m-%dT%H:%M:%S%z")
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', previous_since_tz), "%Y-%m-%d")
        # print('since parameter:', since_parameter)
        previous_since = datetime.strptime(previous_since_tz, "%Y-%m-%dT%H:%M:%S%z") # the %z format code is to indicate timezone as an offset
        
        previous_until = response_json_dict[1]['data'][0]['values'][-1]['end_time']
        previous_until = datetime.strptime(previous_until, "%Y-%m-%dT%H:%M:%S%z")
    except:
        pass
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if (previous_since.date() > since.date()):
        print(f'Fetching new account insights from {datetime.strftime(since, "%Y-%m-%d")} to {datetime.strftime(previous_since, "%Y-%m-%d")}')
        new_insights_df, new_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=since, until=previous_since, since_parameter=since_parameter)
        try:
            df = pd.concat([df, new_insights_df])
        except:
            df = new_insights_df
        new_insights_response_json_dict = dict( # Update the keys of new_insights_response_json_dict before merging with previous dict
            zip([key+len(response_json_dict) for key in new_insights_response_json_dict.keys()], new_insights_response_json_dict.values())
            )
        response_json_dict = {**response_json_dict, **new_insights_response_json_dict}
        
    # if (previous_until.date() < until.date()):

    # if since_parameter:
    #     url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    # else:
    #     url_without_token += f'&since={datetime.timestamp(since)}'
    #     since_parameter = since

    # url = url_without_token+'&access_token='+access_token
    # print(url_without_token)
    
    # response_json_dict = dict()
    # df_list = []
    # earliest_end_time = None
    # page = 1
    # while (since_parameter >= since):
    #     response = requests.get(url)
    #     print(f'Requesting page {page}...')
    #     print('\tResponse status code: ',response.status_code)
    #     response_json_dict[page] = response.json()
    #     if response.status_code//100 != 2: # Stop the function if there is an error in the request
    #         print(response_json_dict[page]['error'])
    #         break
    #     try:
    #         df_list.append(
    #             pd.concat([
    #             json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
    #             json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
    #             ], axis=1)
    #         )
    #     except:
    #         print('No data in request response for page', page)
    #     earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
    #     since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")

    #     try:
    #         next_endpoint = response_json_dict[page]['paging']['previous']
    #         if next_endpoint+access_token != url:
    #             url = next_endpoint
    #         else:
    #             print('end')
    #             break
    #     except: 
    #         break
    #     page +=1
    # try:
    #     df = pd.concat(df_list)
    #     df = df.reset_index(drop=True)
    #     print('Number of days of data:',len(df))
    # except:
    #     df = response
    # if filename:
    #     try:
    #         savepickle(df, filename+'_df', 'sav', csv_path)
    #         savepickle(response_json_dict,filename,'sav',json_path)
    #     except:
    #         print('Unable to save outputs')
    return df, response_json_dict, new_insights_response_json_dict

since = '2023-02-01'
until = '2023-03-01'
filename = 'silvialiftsweights'

data, response_json_dict, new_insights_response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-01', until=until, filename=filename)


Time completed: 2023-03-14 20:51:26.589541
Time completed: 2023-03-14 20:51:26.590545
Fetching new account insights from 2023-01-01 to 2023-01-04
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1672819200.0
since parameter: 2023-01-30 00:00:00
since: 2023-01-01 00:00:00
Requesting page 1...
	Response status code:  200
since_parameter:  2023-01-02 00:00:00
Requesting page 2...
	Response status code:  200
since_parameter:  2022-12-31 00:00:00
Number of days of data: 5


In [25]:
data

,impressions_value,impressions_end_time,reach_value,reach_end_time
0,12382,2023-02-01T08:00:00+0000,4913,2023-02-01T08:00:00+0000
1,11649,2023-02-02T08:00:00+0000,5221,2023-02-02T08:00:00+0000
2,13558,2023-02-03T08:00:00+0000,5155,2023-02-03T08:00:00+0000
3,8538,2023-02-04T08:00:00+0000,4588,2023-02-04T08:00:00+0000
4,12797,2023-02-05T08:00:00+0000,5669,2023-02-05T08:00:00+0000
...,...,...,...,...
0,3,2023-01-02T08:00:00+0000,3,2023-01-02T08:00:00+0000
1,169,2023-01-03T08:00:00+0000,132,2023-01-03T08:00:00+0000
2,141,2023-01-04T08:00:00+0000,103,2023-01-04T08:00:00+0000
3,18,2022-12-31T08:00:00+0000,17,2022-12-31T08:00:00+0000


## Version 6
Adds insights from older dates

In [53]:
def get_ig_account_insights(ig_user_id, access_token, since=None, until=None, 
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    2023-03-02 16:13
    Get the daily impressions and reach a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    else:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since + timedelta(days=1)

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    print('since parameter:', since_parameter)
    print('since:', since)
    while (since_parameter > since):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
                json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
        earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")
        print('since_parameter: ',since_parameter)

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        print('Number of days of data:',len(df))
    except:
        df = df_list 
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

In [54]:
since = '2023-01-01'
until = '2023-03-01'
filename = 'silvialiftsweights'

data2, response_json_dict2 = get_ig_account_insights(
    ig_user_id, access_token, since=since, until=until, filename=None)
data2

https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1677657600.0&since=1675065600.0
since parameter: 2023-01-30 00:00:00
since: 2023-01-01 00:00:00
Requesting page 1...
	Response status code:  200
since_parameter:  2023-01-30 00:00:00
Requesting page 2...
	Response status code:  200
since_parameter:  2022-12-31 00:00:00
Number of days of data: 61


,impressions_value,impressions_end_time,reach_value,reach_end_time
0,196,2023-01-30T08:00:00+0000,141,2023-01-30T08:00:00+0000
1,45,2023-01-31T08:00:00+0000,43,2023-01-31T08:00:00+0000
2,14,2023-02-01T08:00:00+0000,9,2023-02-01T08:00:00+0000
3,5,2023-02-02T08:00:00+0000,4,2023-02-02T08:00:00+0000
4,169,2023-02-03T08:00:00+0000,109,2023-02-03T08:00:00+0000
...,...,...,...,...
56,0,2023-01-25T08:00:00+0000,0,2023-01-25T08:00:00+0000
57,2,2023-01-26T08:00:00+0000,2,2023-01-26T08:00:00+0000
58,38,2023-01-27T08:00:00+0000,1,2023-01-27T08:00:00+0000
59,29,2023-01-28T08:00:00+0000,1,2023-01-28T08:00:00+0000


In [55]:
data2.sort_values(['impressions_end_time'])

,impressions_value,impressions_end_time,reach_value,reach_end_time
31,18,2022-12-31T08:00:00+0000,17,2022-12-31T08:00:00+0000
32,18,2023-01-01T08:00:00+0000,4,2023-01-01T08:00:00+0000
33,3,2023-01-02T08:00:00+0000,3,2023-01-02T08:00:00+0000
34,169,2023-01-03T08:00:00+0000,132,2023-01-03T08:00:00+0000
35,141,2023-01-04T08:00:00+0000,103,2023-01-04T08:00:00+0000
...,...,...,...,...
26,1,2023-02-25T08:00:00+0000,1,2023-02-25T08:00:00+0000
27,32,2023-02-26T08:00:00+0000,9,2023-02-26T08:00:00+0000
28,109,2023-02-27T08:00:00+0000,95,2023-02-27T08:00:00+0000
29,29,2023-02-28T08:00:00+0000,28,2023-02-28T08:00:00+0000


In [56]:
# Make GET request only for dates that have not been saved in previous queries
from datetime import time, datetime, timedelta
def update_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    SH 2023-03-09 12:37
    Get the daily impressions and reach a given Instagram account. Load results that were previously saved.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    if filename:
        filename = f'{filename}_account_insights'
    try:
        df = loadpickle(filename+'.sav', csv_path)
        response_json_dict = loadpickle(filename+'.sav', json_path)
        last_json_page = max(response_json_dict.keys())
        previous_since_tz = response_json_dict[last_json_page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', previous_since_tz), "%Y-%m-%d")
        previous_since = datetime.strptime(previous_since_tz, "%Y-%m-%dT%H:%M:%S%z") # the %z format code is to indicate timezone as an offset
        
        previous_until = response_json_dict[1]['data'][0]['values'][-1]['end_time']
        previous_until = datetime.strptime(previous_until, "%Y-%m-%dT%H:%M:%S%z")
    except:
        pass
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if (previous_since.date() > since.date()):
        print(f'Fetching new account insights from {datetime.strftime(since, "%Y-%m-%d")} to {datetime.strftime(previous_since, "%Y-%m-%d")}')
        new_insights_df, new_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=since, until=previous_since)
        try:
            df = pd.concat([df, new_insights_df])
        except:
            df = new_insights_df
        new_insights_response_json_dict = dict( # Update the keys of new_insights_response_json_dict before merging with previous dict
            zip([key+len(response_json_dict) for key in new_insights_response_json_dict.keys()], new_insights_response_json_dict.values())
            )
        response_json_dict = {**response_json_dict, **new_insights_response_json_dict}
        
    # if filename:
    #     try:
    #         savepickle(df, filename+'_df', 'sav', csv_path)
    #         savepickle(response_json_dict,filename,'sav',json_path)
    #     except:
    #         print('Unable to save outputs')
    return df, response_json_dict, new_insights_response_json_dict

since = '2023-02-01'
until = '2023-03-01'
filename = 'silvialiftsweights'

data, response_json_dict, new_insights_response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-01', until=until, filename=filename)


Time completed: 2023-03-14 21:40:53.728313
Time completed: 2023-03-14 21:40:53.733351
Fetching new account insights from 2023-01-01 to 2023-01-04
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1672819200.0&since=1672560000.0
since parameter: 2023-01-02 00:00:00
since: 2023-01-01 00:00:00
Requesting page 1...
	Response status code:  200
since_parameter:  2023-01-01 00:00:00
Number of days of data: 4


## Version 7
Pull insights from more recent dates than what was previously saved

In [ ]:
def get_ig_account_insights(ig_user_id, access_token, since=None, until=None, 
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    2023-03-02 16:13
    Get the daily impressions and reach a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    else:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since + timedelta(days=1)

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    print('since parameter:', since_parameter)
    print('since:', since)
    while (since_parameter > since):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
                json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
        earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")
        print('since_parameter: ',since_parameter)

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        print('Number of days of data:',len(df))
    except:
        df = df_list 
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

In [60]:
# Make GET request only for dates that have not been saved in previous queries
from datetime import time, datetime, timedelta
def update_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    SH 2023-03-09 12:37
    Get the daily impressions and reach a given Instagram account. Load results that were previously saved.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    if filename:
        filename = f'{filename}_account_insights'
    try:
        df = loadpickle(filename+'.sav', csv_path)
        response_json_dict = loadpickle(filename+'.sav', json_path)
        last_json_page = max(response_json_dict.keys())
        previous_since_tz = response_json_dict[last_json_page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', previous_since_tz), "%Y-%m-%d")
        previous_since = datetime.strptime(previous_since_tz, "%Y-%m-%dT%H:%M:%S%z") # the %z format code is to indicate timezone as an offset
        
        previous_until = response_json_dict[1]['data'][0]['values'][-1]['end_time']
        previous_until = datetime.strptime(previous_until, "%Y-%m-%dT%H:%M:%S%z")
    except:
        pass
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if (previous_since.date() > since.date()):
        print(f'Fetching older account insights from {datetime.strftime(since, "%Y-%m-%d")} to {datetime.strftime(previous_since, "%Y-%m-%d")}')
        older_insights_df, older_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=since, until=previous_since)
        try:
            df = pd.concat([df, older_insights_df])
        except:
            df = older_insights_df
        older_insights_response_json_dict = dict( # Update the keys of older_insights_response_json_dict before merging with previous dict
            zip([key+len(response_json_dict) for key in older_insights_response_json_dict.keys()], older_insights_response_json_dict.values())
            )
        response_json_dict = {**response_json_dict, **older_insights_response_json_dict}
    if (previous_until.date() < until.date()):
        print(f'Fetching newer account insights from {datetime.strftime(previous_until, "%Y-%m-%d")} to {datetime.strftime(until, "%Y-%m-%d")}')
        new_insights_df, new_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=previous_until, until=until)
        try:
            df = pd.concat([df, new_insights_df])
        except:
            df = new_insights_df
        new_insights_response_json_dict = dict( # Update the keys of new_insights_response_json_dict before merging with previous dict
            zip([key+len(response_json_dict) for key in new_insights_response_json_dict.keys()], new_insights_response_json_dict.values())
            )
        response_json_dict = {**response_json_dict, **new_insights_response_json_dict}

        
    # if filename:
    #     try:
    #         savepickle(df, filename+'_df', 'sav', csv_path)
    #         savepickle(response_json_dict,filename,'sav',json_path)
    #     except:
    #         print('Unable to save outputs')
    return df, response_json_dict, older_insights_response_json_dict

since = '2023-02-01'
until = '2023-03-01'
filename = 'silvialiftsweights'

data, response_json_dict, new_insights_response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-01', until='2023-03-05', filename=filename)


Time completed: 2023-03-14 21:53:40.632006
Time completed: 2023-03-14 21:53:40.632006
Fetching older account insights from 2023-01-01 to 2023-01-04
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1672819200.0&since=1672560000.0
since parameter: 2023-01-02 00:00:00
since: 2023-01-01 00:00:00
Requesting page 1...
	Response status code:  200
since_parameter:  2023-01-01 00:00:00
Number of days of data: 4
Fetching newer account insights from 2023-03-01 to 2023-03-05
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1678003200.0&since=1677657600.0
since parameter: 2023-03-02 00:00:00
since: 2023-03-01 00:00:00
Requesting page 1...
	Response status code:  200
since_parameter:  2023-03-01 00:00:00
Number of days of data: 5


In [62]:
data.head()

,impressions_value,impressions_end_time,reach_value,reach_end_time
0,12382,2023-02-01T08:00:00+0000,4913,2023-02-01T08:00:00+0000
1,11649,2023-02-02T08:00:00+0000,5221,2023-02-02T08:00:00+0000
2,13558,2023-02-03T08:00:00+0000,5155,2023-02-03T08:00:00+0000
3,8538,2023-02-04T08:00:00+0000,4588,2023-02-04T08:00:00+0000
4,12797,2023-02-05T08:00:00+0000,5669,2023-02-05T08:00:00+0000


In [63]:
data.sort_values('impressions_end_time')

,impressions_value,impressions_end_time,reach_value,reach_end_time
0,18,2023-01-01T08:00:00+0000,4,2023-01-01T08:00:00+0000
1,3,2023-01-02T08:00:00+0000,3,2023-01-02T08:00:00+0000
2,169,2023-01-03T08:00:00+0000,132,2023-01-03T08:00:00+0000
3,141,2023-01-04T08:00:00+0000,103,2023-01-04T08:00:00+0000
29,19138,2023-01-04T08:00:00+0000,7431,2023-01-04T08:00:00+0000
...,...,...,...,...
28,4917,2023-03-01T08:00:00+0000,1650,2023-03-01T08:00:00+0000
1,51,2023-03-02T08:00:00+0000,30,2023-03-02T08:00:00+0000
2,12,2023-03-03T08:00:00+0000,12,2023-03-03T08:00:00+0000
3,12,2023-03-04T08:00:00+0000,8,2023-03-04T08:00:00+0000


## Version 8

In [ ]:
def get_ig_account_insights(ig_user_id, access_token, since=None, until=None, 
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    2023-03-02 16:13
    Get the daily impressions and reach a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    else:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since + timedelta(days=1)

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    # print('since parameter:', since_parameter)
    # print('since:', since)
    while (since_parameter > since):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
                json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
        earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")
        print('since_parameter: ',since_parameter)

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        df = df.sort_values[]
        print('Number of days of data:',len(df))
    except:
        df = df_list 
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

In [64]:
# Make GET request only for dates that have not been saved in previous queries
from datetime import time, datetime, timedelta
def update_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    SH 2023-03-09 12:37
    Get the daily impressions and reach a given Instagram account. Load results that were previously saved.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    if filename:
        filename = f'{filename}_account_insights'
    try:
        df = loadpickle(filename+'.sav', csv_path)
        response_json_dict = loadpickle(filename+'.sav', json_path)
        last_json_page = max(response_json_dict.keys())
        previous_since_tz = response_json_dict[last_json_page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', previous_since_tz), "%Y-%m-%d")
        previous_since = datetime.strptime(previous_since_tz, "%Y-%m-%dT%H:%M:%S%z") # the %z format code is to indicate timezone as an offset
        
        previous_until = response_json_dict[1]['data'][0]['values'][-1]['end_time']
        previous_until = datetime.strptime(previous_until, "%Y-%m-%dT%H:%M:%S%z")
    except:
        pass
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if (previous_since.date() > since.date()):
        print(f'Fetching older account insights from {datetime.strftime(since, "%Y-%m-%d")} to {datetime.strftime(previous_since, "%Y-%m-%d")}')
        older_insights_df, older_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=since, until=previous_since)
        try:
            df = pd.concat([df, older_insights_df])
        except:
            df = older_insights_df
        older_insights_response_json_dict = dict( # Update the keys of older_insights_response_json_dict before merging with previous dict
            zip([key+len(response_json_dict) for key in older_insights_response_json_dict.keys()], older_insights_response_json_dict.values())
            )
        response_json_dict = {**response_json_dict, **older_insights_response_json_dict}
    if (previous_until.date() < until.date()):
        print(f'Fetching newer account insights from {datetime.strftime(previous_until, "%Y-%m-%d")} to {datetime.strftime(until, "%Y-%m-%d")}')
        new_insights_df, new_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=previous_until, until=until)
        try:
            df = pd.concat([df, new_insights_df])
        except:
            df = new_insights_df
        new_insights_response_json_dict = dict( # Update the keys of new_insights_response_json_dict before merging with previous dict
            zip([key+len(response_json_dict) for key in new_insights_response_json_dict.keys()], new_insights_response_json_dict.values())
            )
        response_json_dict = {**response_json_dict, **new_insights_response_json_dict}
        
    if filename:
        try:
            savepickle(df, filename+'_df', 'sav', csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict, older_insights_response_json_dict

since = '2023-02-01'
until = '2023-03-01'
filename = 'silvialiftsweights'

data, response_json_dict, new_insights_response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-01', until='2023-03-05', filename=filename)


Time completed: 2023-03-14 22:02:13.391454
Time completed: 2023-03-14 22:02:13.392455
Fetching older account insights from 2023-01-01 to 2023-01-04
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1672819200.0&since=1672560000.0
since parameter: 2023-01-02 00:00:00
since: 2023-01-01 00:00:00
Requesting page 1...
	Response status code:  200
since_parameter:  2023-01-01 00:00:00
Number of days of data: 4
Fetching newer account insights from 2023-03-01 to 2023-03-05
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1678003200.0&since=1677657600.0
since parameter: 2023-03-02 00:00:00
since: 2023-03-01 00:00:00
Requesting page 1...
	Response status code:  200
since_parameter:  2023-03-01 00:00:00
Number of days of data: 5
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialif

In [65]:
since = '2023-02-01'
until = '2023-03-01'
filename = 'silvialiftsweights'

data, response_json_dict, new_insights_response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-01', until='2023-03-05', filename=filename)

## There should not have been any API calls here

Time completed: 2023-03-14 22:03:17.946507
Time completed: 2023-03-14 22:03:17.964566
Fetching older account insights from 2023-01-01 to 2023-03-01
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1677657600.0&since=1675065600.0
since parameter: 2023-01-30 00:00:00
since: 2023-01-01 00:00:00
Requesting page 1...
	Response status code:  200
since_parameter:  2023-01-30 00:00:00
Requesting page 2...
	Response status code:  200
since_parameter:  2022-12-31 00:00:00
Number of days of data: 61
Fetching newer account insights from 2023-03-01 to 2023-03-05
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1678003200.0&since=1677657600.0
since parameter: 2023-03-02 00:00:00
since: 2023-03-01 00:00:00
Requesting page 1...
	Response status code:  200
since_parameter:  2023-03-01 00:00:00
Number of days of data: 5
File saved:  C:/Users/silvh/

In [66]:
data.sort_values('impressions_end_time')

,impressions_value,impressions_end_time,reach_value,reach_end_time
31,18,2022-12-31T08:00:00+0000,17,2022-12-31T08:00:00+0000
32,18,2023-01-01T08:00:00+0000,4,2023-01-01T08:00:00+0000
33,3,2023-01-02T08:00:00+0000,3,2023-01-02T08:00:00+0000
34,169,2023-01-03T08:00:00+0000,132,2023-01-03T08:00:00+0000
29,19138,2023-01-04T08:00:00+0000,7431,2023-01-04T08:00:00+0000
...,...,...,...,...
0,289,2023-03-01T08:00:00+0000,170,2023-03-01T08:00:00+0000
1,51,2023-03-02T08:00:00+0000,30,2023-03-02T08:00:00+0000
2,12,2023-03-03T08:00:00+0000,12,2023-03-03T08:00:00+0000
3,12,2023-03-04T08:00:00+0000,8,2023-03-04T08:00:00+0000


## Version 9

In [97]:
def get_ig_account_insights(ig_user_id, access_token, since=None, until=None, 
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    2023-03-02 16:13
    Get the daily impressions and reach a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    else:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since + timedelta(days=1)

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    # print('since parameter:', since_parameter)
    # print('since:', since)
    while (since_parameter > since):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
                json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
        earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")
        print('since_parameter: ',since_parameter)

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        print('Number of days of data:',len(df))
    except:
        df = df_list 
    if filename:
        filename += '_account_insights'
        try:
            savepickle(response_json_dict,filename+'_df','sav',csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

In [98]:
since = '2023-02-01'
until = '2023-03-01'
filename2 = 'silvialiftsweights_03-14'
data, response_json_dict = get_ig_account_insights(
    ig_user_id, access_token, since=since, until=until, filename=filename2)

https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1677657600.0&since=1675238400.0
Requesting page 1...
	Response status code:  200
since_parameter:  2023-02-01 00:00:00
Number of days of data: 29
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-14_account_insights_df.sav
Time completed: 2023-03-14 22:42:30.074136
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-14_account_insights.sav
Time completed: 2023-03-14 22:42:30.075091


In [103]:
# Make GET request only for dates that have not been saved in previous queries
from datetime import time, datetime, timedelta
def update_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    SH 2023-03-09 12:37
    Get the daily impressions and reach a given Instagram account. Load results that were previously saved.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    if filename:
        filename = f'{filename}_account_insights'
    try:
        df = loadpickle(filename+'_df.sav', csv_path)
        response_json_dict = loadpickle(filename+'.sav', json_path)
        last_json_page = max(response_json_dict.keys())
        previous_since_tz = response_json_dict[last_json_page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', previous_since_tz), "%Y-%m-%d")
        previous_since = datetime.strptime(previous_since_tz, "%Y-%m-%dT%H:%M:%S%z") # the %z format code is to indicate timezone as an offset
        
        previous_until = response_json_dict[1]['data'][0]['values'][-1]['end_time']
        previous_until = datetime.strptime(previous_until, "%Y-%m-%dT%H:%M:%S%z")
    except:
        pass
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if (previous_since.date() > since.date()):
        print(f'Fetching older account insights from {datetime.strftime(since, "%Y-%m-%d")} to {datetime.strftime(previous_since, "%Y-%m-%d")}')
        older_insights_df, older_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=since, until=previous_since)
        try:
            df = pd.concat([df, older_insights_df])
        except:
            df = older_insights_df
        # Update the keys of *response_json_dict* before merging with older_insights_response_json_dict. That way, final 
            # response dictionary always has insights from oldest dates first
        response_json_dict = dict( 
            zip([key+len(older_insights_response_json_dict) for key in response_json_dict.keys()], response_json_dict.values())
            )
        response_json_dict = {**older_insights_response_json_dict, **response_json_dict}
    if (previous_until.date() < until.date()):
        print(f'Fetching newer account insights from {datetime.strftime(previous_until, "%Y-%m-%d")} to {datetime.strftime(until, "%Y-%m-%d")}')
        new_insights_df, new_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=previous_until, until=until)
        try:
            df = pd.concat([df, new_insights_df])
        except:
            df = new_insights_df
        new_insights_response_json_dict = dict( # Update the keys of new_insights_response_json_dict before merging with previous dict
            zip([key+len(response_json_dict) for key in new_insights_response_json_dict.keys()], new_insights_response_json_dict.values())
            )
        response_json_dict = {**response_json_dict, **new_insights_response_json_dict}
        
    if filename:
        try:
            savepickle(df, filename+'_df', 'sav', csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

since = '2023-02-01'
until = '2023-03-01'
filename2 = 'silvialiftsweights_03-14'

data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since=since, until=since, filename=filename2)



Time completed: 2023-03-14 22:46:15.220698
Time completed: 2023-03-14 22:46:15.221700
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-14_account_insights_df.sav
Time completed: 2023-03-14 22:46:15.222703
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-14_account_insights.sav
Time completed: 2023-03-14 22:46:15.223701


In [104]:
data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-01', until=since, filename=filename2)

Time completed: 2023-03-14 22:47:23.185139
Time completed: 2023-03-14 22:47:23.187849
Fetching older account insights from 2023-01-01 to 2023-02-01
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1675238400.0&since=1672646400.0
Requesting page 1...
	Response status code:  200
since_parameter:  2023-01-02 00:00:00
Requesting page 2...
	Response status code:  200
since_parameter:  2022-12-03 00:00:00
Number of days of data: 61
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-14_account_insights_df.sav
Time completed: 2023-03-14 22:47:23.908521
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-14_account_insights.sav
Time completed: 2023-03-14 22:47:23.909485


In [105]:
data.sort_values('impressions_end_time')

,impressions_value,impressions_end_time,reach_value,reach_end_time
31,0,2022-12-03T08:00:00+0000,0,2022-12-03T08:00:00+0000
32,6,2022-12-04T08:00:00+0000,2,2022-12-04T08:00:00+0000
33,3,2022-12-05T08:00:00+0000,3,2022-12-05T08:00:00+0000
34,0,2022-12-06T08:00:00+0000,0,2022-12-06T08:00:00+0000
35,1,2022-12-07T08:00:00+0000,1,2022-12-07T08:00:00+0000
...,...,...,...,...
26,29,2023-01-28T08:00:00+0000,1,2023-01-28T08:00:00+0000
27,5,2023-01-29T08:00:00+0000,4,2023-01-29T08:00:00+0000
28,196,2023-01-30T08:00:00+0000,141,2023-01-30T08:00:00+0000
29,45,2023-01-31T08:00:00+0000,43,2023-01-31T08:00:00+0000


In [106]:
data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-01', until=until, filename=filename2)

Time completed: 2023-03-14 22:49:59.067882
Time completed: 2023-03-14 22:49:59.079885
Fetching older account insights from 2023-01-01 to 2023-02-01
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1675238400.0&since=1672646400.0
Requesting page 1...
	Response status code:  200
since_parameter:  2023-01-02 00:00:00
Requesting page 2...
	Response status code:  200
since_parameter:  2022-12-03 00:00:00
Number of days of data: 61
Fetching newer account insights from 2023-02-01 to 2023-03-01
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1677657600.0&since=1675238400.0
Requesting page 1...
	Response status code:  200
since_parameter:  2023-02-01 00:00:00
Number of days of data: 29
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-14_account_insights_df.sav

In [117]:
data.sort_values('impressions_end_time')

,impressions_value,impressions_end_time,reach_value,reach_end_time
31,0,2022-12-03T08:00:00+0000,0,2022-12-03T08:00:00+0000
31,0,2022-12-03T08:00:00+0000,0,2022-12-03T08:00:00+0000
32,6,2022-12-04T08:00:00+0000,2,2022-12-04T08:00:00+0000
32,6,2022-12-04T08:00:00+0000,2,2022-12-04T08:00:00+0000
33,3,2022-12-05T08:00:00+0000,3,2022-12-05T08:00:00+0000
...,...,...,...,...
24,1,2023-02-25T08:00:00+0000,1,2023-02-25T08:00:00+0000
25,32,2023-02-26T08:00:00+0000,9,2023-02-26T08:00:00+0000
26,109,2023-02-27T08:00:00+0000,95,2023-02-27T08:00:00+0000
27,29,2023-02-28T08:00:00+0000,28,2023-02-28T08:00:00+0000


In [122]:
timestamp_column_suffix = 'end_time'
timestamp_column = data.columns[data.columns.str.contains('_'+timestamp_column_suffix)][0]
data[timestamp_column]

0     2023-01-02T08:00:00+0000
1     2023-01-03T08:00:00+0000
2     2023-01-04T08:00:00+0000
3     2023-01-05T08:00:00+0000
4     2023-01-06T08:00:00+0000
                ...           
24    2023-02-25T08:00:00+0000
25    2023-02-26T08:00:00+0000
26    2023-02-27T08:00:00+0000
27    2023-02-28T08:00:00+0000
28    2023-03-01T08:00:00+0000
Name: impressions_end_time, Length: 151, dtype: object

In [160]:
loadpickle(
    'silvialiftsweights_03-14_account_insights_df.sav', r'C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim'
    ).sort_values('impressions_end_time').reset_index(drop=True)

Time completed: 2023-03-14 23:39:34.172171


,impressions_value,impressions_end_time,reach_value,reach_end_time
0,0,2022-12-03T08:00:00+0000,0,2022-12-03T08:00:00+0000
1,0,2022-12-03T08:00:00+0000,0,2022-12-03T08:00:00+0000
2,6,2022-12-04T08:00:00+0000,2,2022-12-04T08:00:00+0000
3,6,2022-12-04T08:00:00+0000,2,2022-12-04T08:00:00+0000
4,3,2022-12-05T08:00:00+0000,3,2022-12-05T08:00:00+0000
...,...,...,...,...
177,109,2023-02-27T08:00:00+0000,95,2023-02-27T08:00:00+0000
178,29,2023-02-28T08:00:00+0000,28,2023-02-28T08:00:00+0000
179,29,2023-02-28T08:00:00+0000,28,2023-02-28T08:00:00+0000
180,289,2023-03-01T08:00:00+0000,170,2023-03-01T08:00:00+0000


## Version 10

In [ ]:
def get_ig_account_insights(ig_user_id, access_token, since=None, until=None, 
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    2023-03-02 16:13
    Get the daily impressions and reach a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    else:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since + timedelta(days=1)

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    # print('since parameter:', since_parameter)
    # print('since:', since)
    while (since_parameter > since):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
                json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
        earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")
        print('since_parameter: ',since_parameter)

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        print('Number of days of data:',len(df))
    except:
        df = df_list 
    if filename:
        filename += '_account_insights'
        try:
            savepickle(response_json_dict,filename+'_df','sav',csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

In [ ]:
timestamp_column_suffix = 'end_time'
timestamp_column = data.columns[data.columns.str.contains('_'+timestamp_column_suffix)][0]
data[timestamp_column]

In [126]:
# Make GET request only for dates that have not been saved in previous queries
from datetime import time, datetime, timedelta
def update_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    timestamp_column_suffix='end_time', filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    SH 2023-03-09 12:37
    Get the daily impressions and reach a given Instagram account. Load results that were previously saved.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    if filename:
        filename = f'{filename}_account_insights'
    try:
        df = loadpickle(filename+'_df.sav', csv_path).reset_index(drop=True)
        timestamp_column = df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]
        df = df.sort_values(timestamp_column)
        response_json_dict = loadpickle(filename+'.sav', json_path)
        last_json_page = max(response_json_dict.keys())
        # previous_since_tz = response_json_dict[last_json_page]['data'][0]['values'][0]['end_time']
        # previous_since = datetime.strptime(previous_since_tz, "%Y-%m-%dT%H:%M:%S%z") # the %z format code is to indicate timezone as an offset
        previous_until = response_json_dict[1]['data'][0]['values'][-1]['end_time']
        previous_until = datetime.strptime(previous_until, "%Y-%m-%dT%H:%M:%S%z")
        previous_since = datetime.strptime(df.loc[0, timestamp_column], "%Y-%m-%dT%H:%M:%S%z") # the %z format code is to indicate timezone as an offset
    except:
        pass
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    print(df.loc[0, timestamp_column])
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if (previous_since.date() > since.date()):
        print(f'\nFetching older account insights from {datetime.strftime(since, "%Y-%m-%d")} to {datetime.strftime(previous_since, "%Y-%m-%d")}')
        older_insights_df, older_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=since, until=previous_since)
        try:
            df = pd.concat([df, older_insights_df])
        except:
            df = older_insights_df
        # Update the keys of *response_json_dict* before merging with older_insights_response_json_dict. That way, final 
            # response dictionary always has insights from oldest dates first
        response_json_dict = dict( 
            zip([key+len(older_insights_response_json_dict) for key in response_json_dict.keys()], response_json_dict.values())
            )
        response_json_dict = {**older_insights_response_json_dict, **response_json_dict}
    if (previous_until.date() < until.date()):
        print(f'\nFetching newer account insights from {datetime.strftime(previous_until, "%Y-%m-%d")} to {datetime.strftime(until, "%Y-%m-%d")}')
        new_insights_df, new_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=previous_until, until=until)
        try:
            df = pd.concat([df, new_insights_df])
        except:
            df = new_insights_df
        new_insights_response_json_dict = dict( # Update the keys of new_insights_response_json_dict before merging with previous dict
            zip([key+len(response_json_dict) for key in new_insights_response_json_dict.keys()], new_insights_response_json_dict.values())
            )
        response_json_dict = {**response_json_dict, **new_insights_response_json_dict}
        
    if filename:
        try:
            savepickle(df, filename+'_df', 'sav', csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

since = '2023-02-01'
until = '2023-03-01'
filename2 = 'silvialiftsweights_03-14'

data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-01', until=until, filename=filename2)



Time completed: 2023-03-14 23:15:06.661853
Time completed: 2023-03-14 23:15:06.667170
2023-01-02T08:00:00+0000

Fetching older account insights from 2023-01-01 to 2023-01-02
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1672646400.0&since=1672560000.0
Requesting page 1...
	Response status code:  200
since_parameter:  2023-01-01 00:00:00
Number of days of data: 2

Fetching newer account insights from 2023-02-01 to 2023-03-01
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1677657600.0&since=1675238400.0
Requesting page 1...
	Response status code:  200
since_parameter:  2023-02-01 00:00:00
Number of days of data: 29
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-14_account_insights_df.sav
Time completed: 2023-03-14 23:15:07.576549
File saved:  C:/U

In [128]:
data.sort_values('impressions_end_time')

,impressions_value,impressions_end_time,reach_value,reach_end_time
92,0,2022-12-03T08:00:00+0000,0,2022-12-03T08:00:00+0000
31,0,2022-12-03T08:00:00+0000,0,2022-12-03T08:00:00+0000
93,6,2022-12-04T08:00:00+0000,2,2022-12-04T08:00:00+0000
32,6,2022-12-04T08:00:00+0000,2,2022-12-04T08:00:00+0000
33,3,2022-12-05T08:00:00+0000,3,2022-12-05T08:00:00+0000
...,...,...,...,...
26,109,2023-02-27T08:00:00+0000,95,2023-02-27T08:00:00+0000
27,29,2023-02-28T08:00:00+0000,28,2023-02-28T08:00:00+0000
149,29,2023-02-28T08:00:00+0000,28,2023-02-28T08:00:00+0000
150,289,2023-03-01T08:00:00+0000,170,2023-03-01T08:00:00+0000


## Version 11

In [161]:
def get_ig_account_insights(ig_user_id, access_token, since=None, until=None, 
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    2023-03-02 16:13
    Get the daily impressions and reach a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    else:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since + timedelta(days=1)

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    while (since_parameter > since):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
                json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
        earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")
        print('since_parameter: ',since_parameter)

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        print('Number of days of data:',len(df))
    except:
        df = df_list 
    if filename:
        filename += '_account_insights'
        try:
            savepickle(df,filename+'_df','sav',csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

In [162]:
since = '2023-02-01'
until = '2023-03-01'
filename2 = 'silvialiftsweights_03-14_2'
data, response_json_dict = get_ig_account_insights(
    ig_user_id, access_token, since=since, until=until, filename=filename2)

https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1677657600.0&since=1675238400.0
Requesting page 1...
	Response status code:  200
since_parameter:  2023-02-01 00:00:00
Number of days of data: 29
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-14_2_account_insights_df.sav
Time completed: 2023-03-14 23:40:44.290538
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-14_2_account_insights.sav
Time completed: 2023-03-14 23:40:44.291538


In [163]:
data.sort_values('impressions_end_time')

,impressions_value,impressions_end_time,reach_value,reach_end_time
0,14,2023-02-01T08:00:00+0000,9,2023-02-01T08:00:00+0000
1,5,2023-02-02T08:00:00+0000,4,2023-02-02T08:00:00+0000
2,169,2023-02-03T08:00:00+0000,109,2023-02-03T08:00:00+0000
3,68,2023-02-04T08:00:00+0000,51,2023-02-04T08:00:00+0000
4,57,2023-02-05T08:00:00+0000,29,2023-02-05T08:00:00+0000
5,78,2023-02-06T08:00:00+0000,67,2023-02-06T08:00:00+0000
6,266,2023-02-07T08:00:00+0000,208,2023-02-07T08:00:00+0000
7,77,2023-02-08T08:00:00+0000,52,2023-02-08T08:00:00+0000
8,21,2023-02-09T08:00:00+0000,21,2023-02-09T08:00:00+0000
9,3,2023-02-10T08:00:00+0000,3,2023-02-10T08:00:00+0000


In [190]:
# Make GET request only for dates that have not been saved in previous queries
from datetime import time, datetime, timedelta
def update_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    timestamp_column_suffix='end_time', filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    SH 2023-03-09 12:37
    Get the daily impressions and reach a given Instagram account. Load results that were previously saved.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    previous_since, previous_until = None, None
    if filename:
        filename = f'{filename}_account_insights'
        # df = loadpickle(filename+'_df.sav', csv_path)
        # df = df.reset_index(drop=True)
        # timestamp_column = df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]
        # df = df.sort_values(timestamp_column)
        # response_json_dict = loadpickle(filename+'.sav', json_path)
        # last_json_page = max(response_json_dict.keys())
        # previous_since = datetime.strptime(df.loc[0, timestamp_column], "%Y-%m-%dT%H:%M:%S%z") # the %z format code is to indicate timezone as an offset
        # previous_until = datetime.strptime(df.iloc[-1][timestamp_column], "%Y-%m-%dT%H:%M:%S%z")
    try:
        df = loadpickle(filename+'_df.sav', csv_path)
        df = df.reset_index(drop=True)
        timestamp_column = df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]
        df = df.sort_values(timestamp_column)
        response_json_dict = loadpickle(filename+'.sav', json_path)
        last_json_page = max(response_json_dict.keys())
        previous_since = datetime.strptime(df.iloc[0][timestamp_column], "%Y-%m-%dT%H:%M:%S%z") # the %z format code is to indicate timezone as an offset
        previous_until = datetime.strptime(df.iloc[-1][timestamp_column], "%Y-%m-%dT%H:%M:%S%z")
    except:
        pass
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    
    if previous_since == None:
        previous_since = since + timedelta (days=1)
        print('Previous `since` parameter could not be found; default to since + 1.')
    if previous_until == None:
        previous_until = until - timedelta (days=1)
        print('Previous `until` parameter could not be found; default to until - 1.')
    if (previous_since.date() > since.date()):
        print(f'\nFetching older account insights from {datetime.strftime(since, "%Y-%m-%d")} to {datetime.strftime(previous_since, "%Y-%m-%d")}')
        older_insights_df, older_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=since, until=previous_since)
        try:
            df = pd.concat([df, older_insights_df])
        except:
            df = older_insights_df
        # Update the keys of *response_json_dict* before merging with older_insights_response_json_dict. That way, final 
            # response dictionary always has insights from oldest dates first
        response_json_dict = dict( 
            zip([key+len(older_insights_response_json_dict) for key in response_json_dict.keys()], response_json_dict.values())
            )
        response_json_dict = {**older_insights_response_json_dict, **response_json_dict}
    if (previous_until.date() < until.date()):
        print(f'\nFetching newer account insights from {datetime.strftime(previous_until, "%Y-%m-%d")} to {datetime.strftime(until, "%Y-%m-%d")}')
        new_insights_df, new_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=previous_until, until=until)
        try:
            df = pd.concat([df, new_insights_df])
        except:
            df = new_insights_df
        new_insights_response_json_dict = dict( # Update the keys of new_insights_response_json_dict before merging with previous dict
            zip([key+len(response_json_dict) for key in new_insights_response_json_dict.keys()], new_insights_response_json_dict.values())
            )
        response_json_dict = {**response_json_dict, **new_insights_response_json_dict}
        
    if filename:
        try:
            savepickle(df, filename+'_df', 'sav', csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

since = '2023-02-01'
until = '2023-03-01'
filename2 = 'silvialiftsweights_03-14_2'

data2, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since=since, until=until, filename=filename2)


Time completed: 2023-03-15 00:05:54.880704
Time completed: 2023-03-15 00:05:54.882681
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-14_2_account_insights_df.sav
Time completed: 2023-03-15 00:05:54.884684
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-14_2_account_insights.sav
Time completed: 2023-03-15 00:05:54.885687


In [191]:
data2, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-21', until=until, filename=filename2)

Time completed: 2023-03-15 00:06:07.343059
Time completed: 2023-03-15 00:06:07.351097
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-14_2_account_insights_df.sav
Time completed: 2023-03-15 00:06:07.353149
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-14_2_account_insights.sav
Time completed: 2023-03-15 00:06:07.354162


In [194]:
loadpickle(
    'silvialiftsweights_03-14_2_account_insights_df.sav', 
    r'C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim'
    ).reset_index(drop=True).sort_values('impressions_end_time')

Time completed: 2023-03-15 00:07:10.243979


,impressions_value,impressions_end_time,reach_value,reach_end_time
0,23,2023-01-21T08:00:00+0000,15,2023-01-21T08:00:00+0000
1,23,2023-01-21T08:00:00+0000,15,2023-01-21T08:00:00+0000
2,5,2023-01-22T08:00:00+0000,5,2023-01-22T08:00:00+0000
3,5,2023-01-22T08:00:00+0000,5,2023-01-22T08:00:00+0000
4,3,2023-01-23T08:00:00+0000,3,2023-01-23T08:00:00+0000
5,3,2023-01-23T08:00:00+0000,3,2023-01-23T08:00:00+0000
6,3,2023-01-24T08:00:00+0000,3,2023-01-24T08:00:00+0000
7,3,2023-01-24T08:00:00+0000,3,2023-01-24T08:00:00+0000
8,0,2023-01-25T08:00:00+0000,0,2023-01-25T08:00:00+0000
9,0,2023-01-25T08:00:00+0000,0,2023-01-25T08:00:00+0000


In [192]:
data2.sort_values('impressions_end_time')

,impressions_value,impressions_end_time,reach_value,reach_end_time
0,23,2023-01-21T08:00:00+0000,15,2023-01-21T08:00:00+0000
1,23,2023-01-21T08:00:00+0000,15,2023-01-21T08:00:00+0000
2,5,2023-01-22T08:00:00+0000,5,2023-01-22T08:00:00+0000
3,5,2023-01-22T08:00:00+0000,5,2023-01-22T08:00:00+0000
4,3,2023-01-23T08:00:00+0000,3,2023-01-23T08:00:00+0000
5,3,2023-01-23T08:00:00+0000,3,2023-01-23T08:00:00+0000
6,3,2023-01-24T08:00:00+0000,3,2023-01-24T08:00:00+0000
7,3,2023-01-24T08:00:00+0000,3,2023-01-24T08:00:00+0000
8,0,2023-01-25T08:00:00+0000,0,2023-01-25T08:00:00+0000
9,0,2023-01-25T08:00:00+0000,0,2023-01-25T08:00:00+0000


In [195]:
data2, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-21', until='2023-03-02', filename=filename2)
data2.sort_values('impressions_end_time')

Time completed: 2023-03-15 00:07:30.997227
Time completed: 2023-03-15 00:07:31.006230
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-14_2_account_insights_df.sav
Time completed: 2023-03-15 00:07:31.008217
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-14_2_account_insights.sav
Time completed: 2023-03-15 00:07:31.009222


,impressions_value,impressions_end_time,reach_value,reach_end_time
0,23,2023-01-21T08:00:00+0000,15,2023-01-21T08:00:00+0000
1,23,2023-01-21T08:00:00+0000,15,2023-01-21T08:00:00+0000
2,5,2023-01-22T08:00:00+0000,5,2023-01-22T08:00:00+0000
3,5,2023-01-22T08:00:00+0000,5,2023-01-22T08:00:00+0000
4,3,2023-01-23T08:00:00+0000,3,2023-01-23T08:00:00+0000
5,3,2023-01-23T08:00:00+0000,3,2023-01-23T08:00:00+0000
6,3,2023-01-24T08:00:00+0000,3,2023-01-24T08:00:00+0000
7,3,2023-01-24T08:00:00+0000,3,2023-01-24T08:00:00+0000
8,0,2023-01-25T08:00:00+0000,0,2023-01-25T08:00:00+0000
9,0,2023-01-25T08:00:00+0000,0,2023-01-25T08:00:00+0000


In [196]:
data2, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-21', until='2023-03-05', filename=filename2)
data2.sort_values('impressions_end_time')

Time completed: 2023-03-15 00:07:57.110088
Time completed: 2023-03-15 00:07:57.119088

Fetching newer account insights from 2023-03-02 to 2023-03-05
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1678003200.0&since=1677744000.0
Requesting page 1...
	Response status code:  200
since_parameter:  2023-03-02 00:00:00
Number of days of data: 4
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-14_2_account_insights_df.sav
Time completed: 2023-03-15 00:07:57.546833
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-14_2_account_insights.sav
Time completed: 2023-03-15 00:07:57.547836


,impressions_value,impressions_end_time,reach_value,reach_end_time
0,23,2023-01-21T08:00:00+0000,15,2023-01-21T08:00:00+0000
1,23,2023-01-21T08:00:00+0000,15,2023-01-21T08:00:00+0000
2,5,2023-01-22T08:00:00+0000,5,2023-01-22T08:00:00+0000
3,5,2023-01-22T08:00:00+0000,5,2023-01-22T08:00:00+0000
4,3,2023-01-23T08:00:00+0000,3,2023-01-23T08:00:00+0000
5,3,2023-01-23T08:00:00+0000,3,2023-01-23T08:00:00+0000
6,3,2023-01-24T08:00:00+0000,3,2023-01-24T08:00:00+0000
7,3,2023-01-24T08:00:00+0000,3,2023-01-24T08:00:00+0000
8,0,2023-01-25T08:00:00+0000,0,2023-01-25T08:00:00+0000
9,0,2023-01-25T08:00:00+0000,0,2023-01-25T08:00:00+0000


In [198]:
data2, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-21', until='2023-03-02', filename=filename2)
data2.sort_values('impressions_end_time')

Time completed: 2023-03-15 00:09:04.279704
Time completed: 2023-03-15 00:09:04.287665
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-14_2_account_insights_df.sav
Time completed: 2023-03-15 00:09:04.290678
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-14_2_account_insights.sav
Time completed: 2023-03-15 00:09:04.291667


,impressions_value,impressions_end_time,reach_value,reach_end_time
0,23,2023-01-21T08:00:00+0000,15,2023-01-21T08:00:00+0000
1,23,2023-01-21T08:00:00+0000,15,2023-01-21T08:00:00+0000
2,5,2023-01-22T08:00:00+0000,5,2023-01-22T08:00:00+0000
3,5,2023-01-22T08:00:00+0000,5,2023-01-22T08:00:00+0000
4,3,2023-01-23T08:00:00+0000,3,2023-01-23T08:00:00+0000
5,3,2023-01-23T08:00:00+0000,3,2023-01-23T08:00:00+0000
6,3,2023-01-24T08:00:00+0000,3,2023-01-24T08:00:00+0000
7,3,2023-01-24T08:00:00+0000,3,2023-01-24T08:00:00+0000
8,0,2023-01-25T08:00:00+0000,0,2023-01-25T08:00:00+0000
9,0,2023-01-25T08:00:00+0000,0,2023-01-25T08:00:00+0000


In [199]:
data2, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-11', until='2023-03-02', filename=filename2)
# data2.sort_values('impressions_end_time')

Time completed: 2023-03-15 00:09:45.417498
Time completed: 2023-03-15 00:09:45.435492

Fetching older account insights from 2023-01-11 to 2023-01-21
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1674288000.0&since=1673424000.0
Requesting page 1...
	Response status code:  200
since_parameter:  2023-01-11 00:00:00
Number of days of data: 11
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-14_2_account_insights_df.sav
Time completed: 2023-03-15 00:09:45.770323
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-14_2_account_insights.sav
Time completed: 2023-03-15 00:09:45.771327


In [200]:
data2.sort_values('impressions_end_time')

,impressions_value,impressions_end_time,reach_value,reach_end_time
0,28,2023-01-11T08:00:00+0000,25,2023-01-11T08:00:00+0000
1,7,2023-01-12T08:00:00+0000,5,2023-01-12T08:00:00+0000
2,412,2023-01-13T08:00:00+0000,180,2023-01-13T08:00:00+0000
3,45,2023-01-14T08:00:00+0000,39,2023-01-14T08:00:00+0000
4,12,2023-01-15T08:00:00+0000,11,2023-01-15T08:00:00+0000
...,...,...,...,...
55,51,2023-03-02T08:00:00+0000,30,2023-03-02T08:00:00+0000
54,51,2023-03-02T08:00:00+0000,30,2023-03-02T08:00:00+0000
56,12,2023-03-03T08:00:00+0000,12,2023-03-03T08:00:00+0000
57,12,2023-03-04T08:00:00+0000,8,2023-03-04T08:00:00+0000


In [202]:
data2, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-31', until='2023-03-02', filename=filename2)
data2.sort_values('impressions_end_time')

Time completed: 2023-03-15 00:10:38.321547
Time completed: 2023-03-15 00:10:38.331554
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-14_2_account_insights_df.sav
Time completed: 2023-03-15 00:10:38.333554
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-14_2_account_insights.sav
Time completed: 2023-03-15 00:10:38.334551


,impressions_value,impressions_end_time,reach_value,reach_end_time
0,28,2023-01-11T08:00:00+0000,25,2023-01-11T08:00:00+0000
1,7,2023-01-12T08:00:00+0000,5,2023-01-12T08:00:00+0000
2,412,2023-01-13T08:00:00+0000,180,2023-01-13T08:00:00+0000
3,45,2023-01-14T08:00:00+0000,39,2023-01-14T08:00:00+0000
4,12,2023-01-15T08:00:00+0000,11,2023-01-15T08:00:00+0000
...,...,...,...,...
65,51,2023-03-02T08:00:00+0000,30,2023-03-02T08:00:00+0000
66,51,2023-03-02T08:00:00+0000,30,2023-03-02T08:00:00+0000
67,12,2023-03-03T08:00:00+0000,12,2023-03-03T08:00:00+0000
68,12,2023-03-04T08:00:00+0000,8,2023-03-04T08:00:00+0000


In [197]:
loadpickle(
    'silvialiftsweights_03-14_2_account_insights_df.sav', 
    r'C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim'
    ).reset_index(drop=True).sort_values('impressions_end_time').loc[0, 'impressions_end_time']

Time completed: 2023-03-15 00:08:38.361854


'2023-01-21T08:00:00+0000'

## Version 12

In [221]:
def get_ig_account_insights(ig_user_id, access_token, since=None, until=None, 
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    2023-03-02 16:13
    Get the daily impressions and reach a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    else:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since + timedelta(days=1)

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    while (since_parameter > since):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
                json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
        earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")
        print('since_parameter: ',since_parameter)

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        print('Number of days of data:',len(df))
    except:
        df = df_list 
    if filename:
        filename += '_account_insights'
        try:
            savepickle(df,filename+'_df','sav',csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

In [215]:
# Make GET request only for dates that have not been saved in previous queries
from datetime import time, datetime, timedelta
def update_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    timestamp_column_suffix='end_time', filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    SH 2023-03-09 12:37
    Get the daily impressions and reach a given Instagram account. Load results that were previously saved.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    previous_since, previous_until = None, None
    if filename:
        filename = f'{filename}_account_insights'
    try:
        df = loadpickle(filename+'_df.sav', csv_path)
        df = df.reset_index(drop=True)
        timestamp_column = df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]
        df = df.sort_values(timestamp_column)
        response_json_dict = loadpickle(filename+'.sav', json_path)
        last_json_page = max(response_json_dict.keys())
        previous_since = datetime.strptime(df.iloc[0][timestamp_column], "%Y-%m-%dT%H:%M:%S%z") # the %z format code is to indicate timezone as an offset
        previous_until = datetime.strptime(df.iloc[-1][timestamp_column], "%Y-%m-%dT%H:%M:%S%z")
    except:
        pass
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    
    if (previous_since == None) & (previous_until == None):
        df, response_json_dict = get_ig_account_insights(ig_user_id, access_token, since=since, until=until, filename=filename)
        return df.sort_values(df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]).reset_index(drop=True), response_json_dict
    elif previous_since == None:
        previous_since = since + timedelta (days=1)
        print('Previous `since` parameter could not be found; default to since + 1.')
    elif previous_until == None:
        previous_until = until - timedelta (days=1)
        print('Previous `until` parameter could not be found; default to until - 1.')
    if (previous_since.date() > since.date()):
        print(f'\nFetching older account insights from {datetime.strftime(since, "%Y-%m-%d")} to {datetime.strftime(previous_since, "%Y-%m-%d")}')
        older_insights_df, older_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=since, until=previous_since)
        try:
            df = pd.concat([df, older_insights_df])
        except:
            df = older_insights_df
        try:
            # Update the keys of *response_json_dict* before merging with older_insights_response_json_dict. That way, final 
                # response dictionary always has insights from oldest dates first
            response_json_dict = dict( 
                zip([key+len(older_insights_response_json_dict) for key in response_json_dict.keys()], response_json_dict.values())
                )
            response_json_dict = {**older_insights_response_json_dict, **response_json_dict}
        except:
            response_json_dict = older_insights_response_json_dict
    if (previous_until.date() < until.date()):
        print(f'\nFetching newer account insights from {datetime.strftime(previous_until, "%Y-%m-%d")} to {datetime.strftime(until, "%Y-%m-%d")}')
        new_insights_df, new_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=previous_until, until=until)
        try:
            df = pd.concat([df, new_insights_df])
        except:
            df = new_insights_df
        new_insights_response_json_dict = dict( # Update the keys of new_insights_response_json_dict before merging with previous dict
            zip([key+len(response_json_dict) for key in new_insights_response_json_dict.keys()], new_insights_response_json_dict.values())
            )
        response_json_dict = {**response_json_dict, **new_insights_response_json_dict}
        
    if filename:
        try:
            savepickle(df, filename+'_df', 'sav', csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df.sort_values(df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]).reset_index(drop=True), response_json_dict

since = '2023-02-01'
until = '2023-03-01'
filename3 = 'silvialiftsweights_03-15'

data2, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since=since, until=until, filename=filename3)


https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1677657600.0&since=1675238400.0
Requesting page 1...
	Response status code:  200
since_parameter:  2023-02-01 00:00:00
Number of days of data: 29
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-15_account_insights_account_insights_df.sav
Time completed: 2023-03-15 00:31:41.710214
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-15_account_insights_account_insights.sav
Time completed: 2023-03-15 00:31:41.711216


In [216]:
data2

,impressions_value,impressions_end_time,reach_value,reach_end_time
0,14,2023-02-01T08:00:00+0000,9,2023-02-01T08:00:00+0000
1,5,2023-02-02T08:00:00+0000,4,2023-02-02T08:00:00+0000
2,169,2023-02-03T08:00:00+0000,109,2023-02-03T08:00:00+0000
3,68,2023-02-04T08:00:00+0000,51,2023-02-04T08:00:00+0000
4,57,2023-02-05T08:00:00+0000,29,2023-02-05T08:00:00+0000
5,78,2023-02-06T08:00:00+0000,67,2023-02-06T08:00:00+0000
6,266,2023-02-07T08:00:00+0000,208,2023-02-07T08:00:00+0000
7,77,2023-02-08T08:00:00+0000,52,2023-02-08T08:00:00+0000
8,21,2023-02-09T08:00:00+0000,21,2023-02-09T08:00:00+0000
9,3,2023-02-10T08:00:00+0000,3,2023-02-10T08:00:00+0000


In [217]:
data2, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-01', until=until, filename=filename3)

https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1677657600.0&since=1675065600.0
Requesting page 1...
	Response status code:  200
since_parameter:  2023-01-30 00:00:00
Requesting page 2...
	Response status code:  200
since_parameter:  2022-12-31 00:00:00
Number of days of data: 61
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-15_account_insights_account_insights_df.sav
Time completed: 2023-03-15 00:31:57.793222
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-15_account_insights_account_insights.sav
Time completed: 2023-03-15 00:31:57.794222


In [218]:
data2

,impressions_value,impressions_end_time,reach_value,reach_end_time
0,18,2022-12-31T08:00:00+0000,17,2022-12-31T08:00:00+0000
1,18,2023-01-01T08:00:00+0000,4,2023-01-01T08:00:00+0000
2,3,2023-01-02T08:00:00+0000,3,2023-01-02T08:00:00+0000
3,169,2023-01-03T08:00:00+0000,132,2023-01-03T08:00:00+0000
4,141,2023-01-04T08:00:00+0000,103,2023-01-04T08:00:00+0000
...,...,...,...,...
56,1,2023-02-25T08:00:00+0000,1,2023-02-25T08:00:00+0000
57,32,2023-02-26T08:00:00+0000,9,2023-02-26T08:00:00+0000
58,109,2023-02-27T08:00:00+0000,95,2023-02-27T08:00:00+0000
59,29,2023-02-28T08:00:00+0000,28,2023-02-28T08:00:00+0000


In [220]:
data2, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-01', until=until, filename=filename3)

https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1677657600.0&since=1675065600.0
Requesting page 1...
	Response status code:  200
since_parameter:  2023-01-30 00:00:00
Requesting page 2...
	Response status code:  200
since_parameter:  2022-12-31 00:00:00
Number of days of data: 61
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-15_account_insights_account_insights_df.sav
Time completed: 2023-03-15 00:33:14.475190
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-15_account_insights_account_insights.sav
Time completed: 2023-03-15 00:33:14.476194


## Version 13

In [ ]:
def get_ig_account_insights(ig_user_id, access_token, since=None, until=None, 
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    2023-03-02 16:13
    Get the daily impressions and reach a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    else:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since + timedelta(days=1)

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    while (since_parameter > since):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
                json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
        earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")
        print('since_parameter: ',since_parameter)

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        print('Number of days of data:',len(df))
    except:
        df = df_list 
    if filename:
        filename += '_account_insights'
        try:
            savepickle(df,filename+'_df','sav',csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

In [222]:
# Make GET request only for dates that have not been saved in previous queries
from datetime import time, datetime, timedelta
def update_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    timestamp_column_suffix='end_time', filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    SH 2023-03-09 12:37
    Get the daily impressions and reach a given Instagram account. Load results that were previously saved.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    previous_since, previous_until = None, None
    if filename:
        filename = f'{filename}_account_insights'
    try:
        df = loadpickle(filename+'_df.sav', csv_path)
        df = df.reset_index(drop=True)
        timestamp_column = df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]
        df = df.sort_values(timestamp_column)
        response_json_dict = loadpickle(filename+'.sav', json_path)
        last_json_page = max(response_json_dict.keys())
        previous_since = datetime.strptime(df.iloc[0][timestamp_column], "%Y-%m-%dT%H:%M:%S%z") # the %z format code is to indicate timezone as an offset
        previous_until = datetime.strptime(df.iloc[-1][timestamp_column], "%Y-%m-%dT%H:%M:%S%z")
    except:
        print('Unable to load prior results; making new API calls for entire date range.')
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    
    if (previous_since == None) & (previous_until == None):
        df, response_json_dict = get_ig_account_insights(ig_user_id, access_token, since=since, until=until, filename=filename)
        return df.sort_values(df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]).reset_index(drop=True), response_json_dict
    elif previous_since == None:
        previous_since = since + timedelta (days=1)
        print('Previous `since` parameter could not be found; default to since + 1.')
    elif previous_until == None:
        previous_until = until - timedelta (days=1)
        print('Previous `until` parameter could not be found; default to until - 1.')
    if (previous_since.date() > since.date()):
        print(f'\nFetching older account insights from {datetime.strftime(since, "%Y-%m-%d")} to {datetime.strftime(previous_since, "%Y-%m-%d")}')
        older_insights_df, older_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=since, until=previous_since)
        try:
            df = pd.concat([df, older_insights_df])
        except:
            df = older_insights_df
        try:
            # Update the keys of *response_json_dict* before merging with older_insights_response_json_dict. That way, final 
                # response dictionary always has insights from oldest dates first
            response_json_dict = dict( 
                zip([key+len(older_insights_response_json_dict) for key in response_json_dict.keys()], response_json_dict.values())
                )
            response_json_dict = {**older_insights_response_json_dict, **response_json_dict}
        except:
            response_json_dict = older_insights_response_json_dict
    if (previous_until.date() < until.date()):
        print(f'\nFetching newer account insights from {datetime.strftime(previous_until, "%Y-%m-%d")} to {datetime.strftime(until, "%Y-%m-%d")}')
        new_insights_df, new_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=previous_until, until=until)
        try:
            df = pd.concat([df, new_insights_df])
        except:
            df = new_insights_df
        new_insights_response_json_dict = dict( # Update the keys of new_insights_response_json_dict before merging with previous dict
            zip([key+len(response_json_dict) for key in new_insights_response_json_dict.keys()], new_insights_response_json_dict.values())
            )
        response_json_dict = {**response_json_dict, **new_insights_response_json_dict}
        
    if filename:
        try:
            savepickle(df, filename+'_df', 'sav', csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df.sort_values(df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]).reset_index(drop=True), response_json_dict

since = '2023-02-01'
until = '2023-03-01'
filename3 = 'silvialiftsweights_03-15_2'

data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since=since, until=until, filename=filename3)


Unable to load prior results; making new API calls for entire date range.
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1677657600.0&since=1675238400.0
Requesting page 1...
	Response status code:  200
since_parameter:  2023-02-01 00:00:00
Number of days of data: 29
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-15_2_account_insights_account_insights_df.sav
Time completed: 2023-03-15 00:37:54.319413
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-15_2_account_insights_account_insights.sav
Time completed: 2023-03-15 00:37:54.319413


In [223]:
data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-21', until=until, filename=filename3)

Unable to load prior results; making new API calls for entire date range.
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1677657600.0&since=1675065600.0
Requesting page 1...
	Response status code:  200
since_parameter:  2023-01-30 00:00:00
Requesting page 2...
	Response status code:  200
since_parameter:  2022-12-31 00:00:00
Number of days of data: 61
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-15_2_account_insights_account_insights_df.sav
Time completed: 2023-03-15 00:38:44.612196
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-15_2_account_insights_account_insights.sav
Time completed: 2023-03-15 00:38:44.613198


## Version 14

In [ ]:
def get_ig_account_insights(ig_user_id, access_token, since=None, until=None, 
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    2023-03-02 16:13
    Get the daily impressions and reach a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    else:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since + timedelta(days=1)

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    while (since_parameter > since):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
                json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
        earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")
        print('since_parameter: ',since_parameter)

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        print('Number of days of data:',len(df))
    except:
        df = df_list 
    if filename:
        filename += '_account_insights'
        try:
            savepickle(df,filename+'_df','sav',csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

In [235]:
# Make GET request only for dates that have not been saved in previous queries
from datetime import time, datetime, timedelta
def update_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    timestamp_column_suffix='end_time', filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    SH 2023-03-09 12:37
    Get the daily impressions and reach a given Instagram account. Load results that were previously saved.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    previous_since, previous_until = None, None
    if filename:
        filename2 = f'{filename}_account_insights'
        # df = loadpickle(filename+'_df.sav', csv_path)
        # df = df.reset_index(drop=True)
        # timestamp_column = df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]
        # df = df.sort_values(timestamp_column)
        # response_json_dict = loadpickle(filename+'.sav', json_path)
        # last_json_page = max(response_json_dict.keys())
        # previous_since = datetime.strptime(df.iloc[0][timestamp_column], "%Y-%m-%dT%H:%M:%S%z") # the %z format code is to indicate timezone as an offset
        # previous_until = datetime.strptime(df.iloc[-1][timestamp_column], "%Y-%m-%dT%H:%M:%S%z")
    try:
        df = loadpickle(filename2+'_df.sav', csv_path)
        df = df.reset_index(drop=True)
        timestamp_column = df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]
        df = df.sort_values(timestamp_column)
        response_json_dict = loadpickle(filename2+'.sav', json_path)
        last_json_page = max(response_json_dict.keys())
        previous_since = datetime.strptime(df.iloc[0][timestamp_column], "%Y-%m-%dT%H:%M:%S%z") # the %z format code is to indicate timezone as an offset
        previous_until = datetime.strptime(df.iloc[-1][timestamp_column], "%Y-%m-%dT%H:%M:%S%z")
    except:
        print('Unable to load prior results; making new API calls for entire date range.')
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    
    if (previous_since == None) & (previous_until == None):
        df, response_json_dict = get_ig_account_insights(ig_user_id, access_token, since=since, until=until, filename=filename)
        return df.sort_values(df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]).reset_index(drop=True), response_json_dict
    elif previous_since == None:
        previous_since = since + timedelta (days=1)
        print('Previous `since` parameter could not be found; default to since + 1.')
    elif previous_until == None:
        previous_until = until - timedelta (days=1)
        print('Previous `until` parameter could not be found; default to until - 1.')
    if (previous_since.date() > since.date()):
        print(f'\nFetching older account insights from {datetime.strftime(since, "%Y-%m-%d")} to {datetime.strftime(previous_since, "%Y-%m-%d")}')
        older_insights_df, older_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=since, until=previous_since)
        try:
            df = pd.concat([df.copy(), older_insights_df])
        except:
            df = older_insights_df
        try:
            # Update the keys of *response_json_dict* before merging with older_insights_response_json_dict. That way, final 
                # response dictionary always has insights from oldest dates first
            response_json_dict = dict( 
                zip([key+len(older_insights_response_json_dict) for key in response_json_dict.keys()], response_json_dict.values())
                )
            response_json_dict = {**older_insights_response_json_dict, **response_json_dict}
        except:
            response_json_dict = older_insights_response_json_dict
    if (previous_until.date() < until.date()):
        print(f'\nFetching newer account insights from {datetime.strftime(previous_until, "%Y-%m-%d")} to {datetime.strftime(until, "%Y-%m-%d")}')
        new_insights_df, new_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=previous_until, until=until)
        try:
            df = pd.concat([df.copy(), new_insights_df])
        except:
            df = new_insights_df
        new_insights_response_json_dict = dict( # Update the keys of new_insights_response_json_dict before merging with previous dict
            zip([key+len(response_json_dict) for key in new_insights_response_json_dict.keys()], new_insights_response_json_dict.values())
            )
        response_json_dict = {**response_json_dict, **new_insights_response_json_dict}
        
    if filename:
        try:
            savepickle(df, filename+'_df', 'sav', csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df.sort_values(df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]).reset_index(drop=True), response_json_dict

since = '2023-02-01'
until = '2023-03-01'
filename3 = 'silvialiftsweights_03-15_2'

data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since=since, until=until, filename=filename3)


Time completed: 2023-03-15 00:49:23.009939
Time completed: 2023-03-15 00:49:23.011946
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-15_2_df.sav
Time completed: 2023-03-15 00:49:23.012939
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-15_2.sav
Time completed: 2023-03-15 00:49:23.013939


In [236]:
data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-21', until=until, filename=filename3)

Time completed: 2023-03-15 00:49:28.663266
Time completed: 2023-03-15 00:49:28.666382

Fetching older account insights from 2023-01-21 to 2023-02-01
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1675238400.0&since=1674288000.0
Requesting page 1...
	Response status code:  200
since_parameter:  2023-01-21 00:00:00
Number of days of data: 12
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-15_2_df.sav
Time completed: 2023-03-15 00:49:28.955256
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-15_2.sav
Time completed: 2023-03-15 00:49:28.955256


In [239]:
data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-21', until=until, filename=filename3)

Time completed: 2023-03-15 00:50:38.642267
Time completed: 2023-03-15 00:50:38.643274

Fetching older account insights from 2023-01-21 to 2023-02-01
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1675238400.0&since=1674288000.0
Requesting page 1...
	Response status code:  200
since_parameter:  2023-01-21 00:00:00
Number of days of data: 12
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-15_2_df.sav
Time completed: 2023-03-15 00:50:38.932952
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-15_2.sav
Time completed: 2023-03-15 00:50:38.932952


In [240]:
data

,impressions_value,impressions_end_time,reach_value,reach_end_time
0,23,2023-01-21T08:00:00+0000,15,2023-01-21T08:00:00+0000
1,5,2023-01-22T08:00:00+0000,5,2023-01-22T08:00:00+0000
2,3,2023-01-23T08:00:00+0000,3,2023-01-23T08:00:00+0000
3,3,2023-01-24T08:00:00+0000,3,2023-01-24T08:00:00+0000
4,0,2023-01-25T08:00:00+0000,0,2023-01-25T08:00:00+0000
5,2,2023-01-26T08:00:00+0000,2,2023-01-26T08:00:00+0000
6,38,2023-01-27T08:00:00+0000,1,2023-01-27T08:00:00+0000
7,29,2023-01-28T08:00:00+0000,1,2023-01-28T08:00:00+0000
8,5,2023-01-29T08:00:00+0000,4,2023-01-29T08:00:00+0000
9,196,2023-01-30T08:00:00+0000,141,2023-01-30T08:00:00+0000


In [243]:
loadpickle(
    filename3+'_df.sav', 
    r'C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim'
    ).reset_index(drop=True).sort_values('impressions_end_time')

Time completed: 2023-03-15 00:52:44.280815


,impressions_value,impressions_end_time,reach_value,reach_end_time
29,23,2023-01-21T08:00:00+0000,15,2023-01-21T08:00:00+0000
30,5,2023-01-22T08:00:00+0000,5,2023-01-22T08:00:00+0000
31,3,2023-01-23T08:00:00+0000,3,2023-01-23T08:00:00+0000
32,3,2023-01-24T08:00:00+0000,3,2023-01-24T08:00:00+0000
33,0,2023-01-25T08:00:00+0000,0,2023-01-25T08:00:00+0000
34,2,2023-01-26T08:00:00+0000,2,2023-01-26T08:00:00+0000
35,38,2023-01-27T08:00:00+0000,1,2023-01-27T08:00:00+0000
36,29,2023-01-28T08:00:00+0000,1,2023-01-28T08:00:00+0000
37,5,2023-01-29T08:00:00+0000,4,2023-01-29T08:00:00+0000
38,196,2023-01-30T08:00:00+0000,141,2023-01-30T08:00:00+0000


In [244]:
data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-21', until=until, filename=filename3)

Time completed: 2023-03-15 00:53:18.486406
Time completed: 2023-03-15 00:53:18.488441

Fetching older account insights from 2023-01-21 to 2023-02-01
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1675238400.0&since=1674288000.0
Requesting page 1...
	Response status code:  200
since_parameter:  2023-01-21 00:00:00
Number of days of data: 12
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-15_2_df.sav
Time completed: 2023-03-15 00:53:18.785551
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-15_2.sav
Time completed: 2023-03-15 00:53:18.786553


## Version 15

In [ ]:
def get_ig_account_insights(ig_user_id, access_token, since=None, until=None, 
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    2023-03-02 16:13
    Get the daily impressions and reach a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    else:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since + timedelta(days=1)

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    while (since_parameter > since):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
                json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
        earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")
        print('since_parameter: ',since_parameter)

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        print('Number of days of data:',len(df))
    except:
        df = df_list 
    if filename:
        filename += '_account_insights'
        try:
            savepickle(df,filename+'_df','sav',csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

In [258]:
# Make GET request only for dates that have not been saved in previous queries
from datetime import time, datetime, timedelta
def update_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    timestamp_column_suffix='end_time', filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    SH 2023-03-09 12:37
    Get the daily impressions and reach a given Instagram account. Load results that were previously saved.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    previous_since, previous_until = None, None
    if filename:
        filename2 = f'{filename}_account_insights'
    try:
        df = loadpickle(filename2+'_df.sav', csv_path)
        df = df.reset_index(drop=True)
        timestamp_column = df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]
        df = df.sort_values(timestamp_column)
        response_json_dict = loadpickle(filename2+'.sav', json_path)
        previous_since = datetime.strptime(df.iloc[0][timestamp_column], "%Y-%m-%dT%H:%M:%S%z") # the %z format code is to indicate timezone as an offset
        previous_until = datetime.strptime(df.iloc[-1][timestamp_column], "%Y-%m-%dT%H:%M:%S%z")
        print('previous since date:', previous_since)
        print('previous until date:', previous_until)
    except:
        print('Unable to load prior results; making new API calls for entire date range.')
    
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    
    if (previous_since == None) & (previous_until == None):
        df, response_json_dict = get_ig_account_insights(ig_user_id, access_token, since=since, until=until, filename=filename)
        return df.sort_values(df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]).reset_index(drop=True), response_json_dict
    elif previous_since == None:
        previous_since = since + timedelta (days=1)
        print('Previous `since` parameter could not be found; default to since + 1.')
    elif previous_until == None:
        previous_until = until - timedelta (days=1)
        print('Previous `until` parameter could not be found; default to until - 1.')
    if (previous_since.date() > since.date()):
        print(f'\nFetching older account insights from {datetime.strftime(since, "%Y-%m-%d")} to {datetime.strftime(previous_since, "%Y-%m-%d")}')
        older_insights_df, older_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=since, until=previous_since)
        try:
            df = pd.concat([df.copy(), older_insights_df])
        except:
            df = older_insights_df
        try:
            # Update the keys of *response_json_dict* before merging with older_insights_response_json_dict. That way, final 
                # response dictionary always has insights from oldest dates first
            response_json_dict = dict( 
                zip([key+len(older_insights_response_json_dict) for key in response_json_dict.keys()], response_json_dict.values())
                )
            response_json_dict = {**older_insights_response_json_dict, **response_json_dict}
        except:
            response_json_dict = older_insights_response_json_dict
    if (previous_until.date() < until.date()):
        print(f'\nFetching newer account insights from {datetime.strftime(previous_until, "%Y-%m-%d")} to {datetime.strftime(until, "%Y-%m-%d")}')
        new_insights_df, new_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=previous_until, until=until)
        try:
            df = pd.concat([df.copy(), new_insights_df])
        except:
            df = new_insights_df
        new_insights_response_json_dict = dict( # Update the keys of new_insights_response_json_dict before merging with previous dict
            zip([key+len(response_json_dict) for key in new_insights_response_json_dict.keys()], new_insights_response_json_dict.values())
            )
        response_json_dict = {**response_json_dict, **new_insights_response_json_dict}
    
        
    if filename:
        try:
            savepickle(df, filename2+'_df', 'sav', csv_path)
            savepickle(response_json_dict,filename2,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df.sort_values(df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]).reset_index(drop=True), response_json_dict

since = '2023-02-01'
until = '2023-03-01'
filename3 = 'silvialiftsweights_03-15_3'

data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since=since, until=until, filename=filename3)


Time completed: 2023-03-15 01:07:22.300829
Time completed: 2023-03-15 01:07:22.303828
previous since date: 2023-02-01 08:00:00+00:00
previous until date: 2023-03-01 08:00:00+00:00
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-15_3_account_insights_df.sav
Time completed: 2023-03-15 01:07:22.306826
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-15_3_account_insights.sav
Time completed: 2023-03-15 01:07:22.307825


In [255]:
loadpickle(
    filename3+'_df.sav', 
    r'C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim'
    ).reset_index(drop=True).sort_values('impressions_end_time')

Time completed: 2023-03-15 01:04:24.280137


,impressions_value,impressions_end_time,reach_value,reach_end_time
29,23,2023-01-21T08:00:00+0000,15,2023-01-21T08:00:00+0000
30,5,2023-01-22T08:00:00+0000,5,2023-01-22T08:00:00+0000
31,3,2023-01-23T08:00:00+0000,3,2023-01-23T08:00:00+0000
32,3,2023-01-24T08:00:00+0000,3,2023-01-24T08:00:00+0000
33,0,2023-01-25T08:00:00+0000,0,2023-01-25T08:00:00+0000
34,2,2023-01-26T08:00:00+0000,2,2023-01-26T08:00:00+0000
35,38,2023-01-27T08:00:00+0000,1,2023-01-27T08:00:00+0000
36,29,2023-01-28T08:00:00+0000,1,2023-01-28T08:00:00+0000
37,5,2023-01-29T08:00:00+0000,4,2023-01-29T08:00:00+0000
38,196,2023-01-30T08:00:00+0000,141,2023-01-30T08:00:00+0000


In [259]:
data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-21', until=until, filename=filename3)

Time completed: 2023-03-15 01:07:49.181412
Time completed: 2023-03-15 01:07:49.185402
previous since date: 2023-02-01 08:00:00+00:00
previous until date: 2023-03-01 08:00:00+00:00

Fetching older account insights from 2023-01-21 to 2023-02-01
https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1675238400.0&since=1674288000.0
Requesting page 1...
	Response status code:  200
since_parameter:  2023-01-21 00:00:00
Number of days of data: 12
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-15_3_account_insights_df.sav
Time completed: 2023-03-15 01:07:49.560935
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-15_3_account_insights.sav
Time completed: 2023-03-15 01:07:49.560935


In [260]:
data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-21', until=until, filename=filename3)

Time completed: 2023-03-15 01:08:01.075077
Time completed: 2023-03-15 01:08:01.078759
previous since date: 2023-01-21 08:00:00+00:00
previous until date: 2023-03-01 08:00:00+00:00
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-15_3_account_insights_df.sav
Time completed: 2023-03-15 01:08:01.080785
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-15_3_account_insights.sav
Time completed: 2023-03-15 01:08:01.081790


In [263]:
data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-21', until='2023-03-03', filename=filename3)

Time completed: 2023-03-15 01:10:06.833010
Time completed: 2023-03-15 01:10:06.835046
previous since date: 2023-01-21 08:00:00+00:00
previous until date: 2023-03-03 08:00:00+00:00
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-15_3_account_insights_df.sav
Time completed: 2023-03-15 01:10:06.837010
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-15_3_account_insights.sav
Time completed: 2023-03-15 01:10:06.838015


In [262]:
data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-21', until='2023-03-03', filename=filename3)

Time completed: 2023-03-15 01:08:49.410722
Time completed: 2023-03-15 01:08:49.428698
previous since date: 2023-01-21 08:00:00+00:00
previous until date: 2023-03-03 08:00:00+00:00
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-15_3_account_insights_df.sav
Time completed: 2023-03-15 01:08:49.431701
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-15_3_account_insights.sav
Time completed: 2023-03-15 01:08:49.432697


## Version 16

In [ ]:
def get_ig_account_insights(ig_user_id, access_token, since=None, until=None, 
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    2023-03-02 16:13
    Get the daily impressions and reach a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    else:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since + timedelta(days=1)

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    while (since_parameter > since):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
                json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
        earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")
        print('since_parameter: ',since_parameter)

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        print('Number of days of data:',len(df))
    except:
        df = df_list 
    if filename:
        filename += '_account_insights'
        try:
            savepickle(df,filename+'_df','sav',csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

In [265]:
# Make GET request only for dates that have not been saved in previous queries
from datetime import time, datetime, timedelta
def update_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    timestamp_column_suffix='end_time', filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    2023-03-15 1:22
    Get the daily impressions and reach a given Instagram account. 
    Load any results that were previously saved (pull new data if no previously saved results available).

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - timestamp_column_suffix (str): Suffix of the timestamp columns. default is 'end_time'. 
            Required to parse out the date range of the previously saved outputs.
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    previous_since, previous_until = None, None
    if filename:
        filename2 = f'{filename}_account_insights'
    try:
        df = loadpickle(filename2+'_df.sav', csv_path)
        df = df.reset_index(drop=True)
        timestamp_column = df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]
        df = df.sort_values(timestamp_column)
        response_json_dict = loadpickle(filename2+'.sav', json_path)
        previous_since = datetime.strptime(df.iloc[0][timestamp_column], "%Y-%m-%dT%H:%M:%S%z") # the %z format code is to indicate timezone as an offset
        previous_until = datetime.strptime(df.iloc[-1][timestamp_column], "%Y-%m-%dT%H:%M:%S%z")
        print('previous since date:', previous_since)
        print('previous until date:', previous_until)
    except:
        print('Unable to load prior results; making new API calls for entire date range.')
    
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    
    if (previous_since == None) & (previous_until == None):
        df, response_json_dict = get_ig_account_insights(ig_user_id, access_token, since=since, until=until, filename=filename)
        return df.sort_values(df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]).reset_index(drop=True), response_json_dict
    elif previous_since == None:
        previous_since = since + timedelta (days=1)
        print('Previous `since` parameter could not be found; default to since + 1.')
    elif previous_until == None:
        previous_until = until - timedelta (days=1)
        print('Previous `until` parameter could not be found; default to until - 1.')
    if (previous_since.date() > since.date()):
        print(f'\nFetching older account insights from {datetime.strftime(since, "%Y-%m-%d")} to {datetime.strftime(previous_since, "%Y-%m-%d")}')
        older_insights_df, older_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=since, until=previous_since)
        try:
            df = pd.concat([df.copy(), older_insights_df])
        except:
            df = older_insights_df
        try:
            # Update the keys of *response_json_dict* before merging with older_insights_response_json_dict. That way, final 
                # response dictionary always has insights from oldest dates first
            response_json_dict = dict( 
                zip([key+len(older_insights_response_json_dict) for key in response_json_dict.keys()], response_json_dict.values())
                )
            response_json_dict = {**older_insights_response_json_dict, **response_json_dict}
        except:
            response_json_dict = older_insights_response_json_dict
    if (previous_until.date() < until.date()):
        print(f'\nFetching newer account insights from {datetime.strftime(previous_until, "%Y-%m-%d")} to {datetime.strftime(until, "%Y-%m-%d")}')
        new_insights_df, new_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=previous_until, until=until)
        try:
            df = pd.concat([df.copy(), new_insights_df])
        except:
            df = new_insights_df
        new_insights_response_json_dict = dict( # Update the keys of new_insights_response_json_dict before merging with previous dict
            zip([key+len(response_json_dict) for key in new_insights_response_json_dict.keys()], new_insights_response_json_dict.values())
            )
        response_json_dict = {**response_json_dict, **new_insights_response_json_dict}
    
    if (previous_until.date() >= until.date()) & (previous_since.date() <= since.date()):
        print('\nLoading previous saved results; no new API calls required.\n')
        
    if filename:
        try:
            savepickle(df, filename2+'_df', 'sav', csv_path)
            savepickle(response_json_dict,filename2,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df.sort_values(df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]).reset_index(drop=True), response_json_dict

since = '2023-02-01'
until = '2023-03-01'
filename3 = 'silvialiftsweights_03-15_3'

data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since=since, until=until, filename=filename3)


Time completed: 2023-03-15 01:12:54.934493
Time completed: 2023-03-15 01:12:54.942491
previous since date: 2023-01-21 08:00:00+00:00
previous until date: 2023-03-03 08:00:00+00:00

Loading previous saved results; no new API calls required.

File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-15_3_account_insights_df.sav
Time completed: 2023-03-15 01:12:54.944500
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-15_3_account_insights.sav
Time completed: 2023-03-15 01:12:54.946496


In [266]:
data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-21', until=until, filename=filename3)

Time completed: 2023-03-15 01:13:04.954982
Time completed: 2023-03-15 01:13:04.958981
previous since date: 2023-01-21 08:00:00+00:00
previous until date: 2023-03-03 08:00:00+00:00

Loading previous saved results; no new API calls required.

File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-15_3_account_insights_df.sav
Time completed: 2023-03-15 01:13:04.961982
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-15_3_account_insights.sav
Time completed: 2023-03-15 01:13:04.962982


In [267]:
data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-21', until=until, filename=filename3)

Time completed: 2023-03-15 01:13:07.867489
Time completed: 2023-03-15 01:13:07.869484
previous since date: 2023-01-21 08:00:00+00:00
previous until date: 2023-03-03 08:00:00+00:00

Loading previous saved results; no new API calls required.

File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-15_3_account_insights_df.sav
Time completed: 2023-03-15 01:13:07.871483
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-15_3_account_insights.sav
Time completed: 2023-03-15 01:13:07.872483


In [268]:
data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-21', until='2023-03-03', filename=filename3)

Time completed: 2023-03-15 01:13:11.001121
Time completed: 2023-03-15 01:13:11.004121
previous since date: 2023-01-21 08:00:00+00:00
previous until date: 2023-03-03 08:00:00+00:00

Loading previous saved results; no new API calls required.

File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-15_3_account_insights_df.sav
Time completed: 2023-03-15 01:13:11.006128
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-15_3_account_insights.sav
Time completed: 2023-03-15 01:13:11.007130


In [269]:
data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-21', until='2023-03-03', filename=filename3)

Time completed: 2023-03-15 01:13:13.134806
Time completed: 2023-03-15 01:13:13.136804
previous since date: 2023-01-21 08:00:00+00:00
previous until date: 2023-03-03 08:00:00+00:00

Loading previous saved results; no new API calls required.

File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_03-15_3_account_insights_df.sav
Time completed: 2023-03-15 01:13:13.138798
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_03-15_3_account_insights.sav
Time completed: 2023-03-15 01:13:13.139798


# *End of Page*